In [ ]:
# pip freeze > req

In [ ]:
ls ./data

In [ ]:
import dask.dataframe as dd
from py2neo import Graph
from tqdm.auto import tqdm
import time

In [ ]:
graph = Graph("bolt://neo4j:7687", auth=("neo4j", "qwer1234"))

In [ ]:
ddf = dd.read_csv('data/conceptnet-assertions-5.7.0.csv', header=None, sep='\t', names=['uri','relation','node_start','node_end','info'])
ddf = ddf[ddf['node_start'].str.startswith('/c/en') & ddf['node_end'].str.startswith('/c/en')]
ddf

In [ ]:
def create_conceptnet(df):
    if df.shape[0] < 3:
        return None
    df['relation'] = df['relation'].apply(lambda x: x.split('/')[-1])
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        tx = graph.begin()
        tx.run(f"""
            MERGE (head:conceptNet {{name:"{row.get('node_start',"")}"}})
            MERGE (tail:conceptNet {{name:"{row.get('node_end',"")}"}})
            MERGE (head)-[:{row.get('relation',"")} {{name: "{row.get('relation',"")}"}}]->(tail)
        """)
        graph.commit(tx)
        if index%1000==0:
            time.sleep(10)

In [ ]:
ddf.map_partitions(lambda df: create_conceptnet(df)).compute()